In [1]:
import pandas as pd 
import numpy as np 

from datetime import date

In [2]:
month_dict = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
              'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10',
              'Nov': '11', 'Dec': '12'}

def convertStringIntoDate(newDate):
    #print(newDate)
    newDate = [x.strip() for x in newDate.split(" ")]
    newDate[0] = int(newDate[0])
    newDate[1] = int(month_dict[newDate[1]])
    newDate[2] = int(newDate[2])
    return date(newDate[2],newDate[1],newDate[0])

def convertDateToInt(currDate, minDate):
    return (currDate - minDate).days

In [3]:
runner_df = pd.read_csv("data/raw/runners_UK2.csv").drop("Unnamed: 0", axis=1)
race_df = pd.read_csv("data/raw/races_UK2.csv").drop("Unnamed: 0", axis=1)

new_data = pd.read_csv("data/raw/full_data4.csv")
new_data.reset_index(inplace=True)
new_data = new_data.rename(columns={"index":"race_id","Unnamed: 0":"horse_ids","male_pedigree":"sire_id",
                                   "female_pedigree":"dam_id", "older_pedigree":"dam_sire_id"})

In [13]:
race_df['date']

0          2 Jan 2010
1          2 Jan 2010
2          2 Jan 2010
3          2 Jan 2010
4          2 Jan 2010
             ...     
217000    19 Apr 2021
217001    19 Apr 2021
217002    19 Apr 2021
217003    19 Apr 2021
217004    19 Apr 2021
Name: date, Length: 217005, dtype: object

In [5]:
df = race_df.merge(runner_df, on="race_id")
df = df.merge(new_data[['race_id','horse_ids','sire_id','dam_id','dam_sire_id']], on=["race_id","horse_ids"])
df['date'] = df['date'].apply(lambda row: convertStringIntoDate(row))

In [6]:
df.to_csv("data/prep.csv")

In [7]:
def compute_horse_features(df, group_col="track"):
    df = df.sort_values('date')
    
    df['days_since_last_race'] = df['date'] - df.groupby('horse_ids')['date'].shift()
    
    return df

df2 = compute_horse_features(df)

In [8]:
df2.loc[df.horse_ids == 1370464]

,race_id,track_name,track_id,race_class,distance,going,race_age,race_handicap,race_type,date,...,best_figures_dist,best_figures_going,top_speeds,ratings,official_ratings,odds,places,sire_id,dam_id,dam_sire_id
1908886,689345.0,Cork (IRE),596.0,NaN,3520.0,Soft To Heavy,4yo,False,Hurdle,2017-11-19,...,NaN,NaN,71,121,–,11.00,4,462135,906197,49160
2005099,701697.0,Kilbeggan (IRE),203.0,NaN,3520.0,Soft,4-5yo,False,Hurdle,2018-05-11,...,NaN,NaN,83,118,–,5.50,1,462135,906197,49160
2072706,708384.0,Galway (IRE),183.0,NaN,4510.0,Soft,4yo+,False,Hurdle,2018-08-02,...,NaN,NaN,67,130,–,8.50,4,462135,906197,49160
2114668,713227.0,Tipperary (IRE),202.0,NaN,3520.0,Good,4yo+,True,Hurdle,2018-10-07,...,NaN,NaN,56,115,125,4.50,7,462135,906197,49160
2169248,718038.0,Punchestown (IRE),195.0,NaN,3520.0,Good,4yo+,False,Hurdle,2018-12-09,...,NaN,NaN,105,124,124,11.00,2,462135,906197,49160
2178323,719532.0,Leopardstown (IRE),187.0,NaN,4400.0,Good To Yielding,4yo+,True,Hurdle,2018-12-29,...,NaN,NaN,71,122,124,8.00,8,462135,906197,49160
2192028,720945.0,Navan (IRE),193.0,NaN,3520.0,Yielding,4yo+,False,Hurdle,2019-01-19,...,NaN,NaN,96,124,123,3.50,3,462135,906197,49160
2283280,733185.0,Punchestown (IRE),195.0,NaN,4620.0,Good To Yielding,4yo+,False,Chase,2019-06-12,...,NaN,NaN,34,59,–,8.00,9,462135,906197,49160
2312215,735682.0,Kilbeggan (IRE),203.0,NaN,4840.0,Yielding,5yo+,False,Chase,2019-07-19,...,NaN,NaN,115,130,–,17.00,2,462135,906197,49160
2322167,736568.0,Galway (IRE),183.0,NaN,4950.0,Good,5yo+,False,Chase,2019-08-01,...,NaN,NaN,80,126,–,9.00,4,462135,906197,49160


In [14]:
new_data

,race_id,horse_ids,length,pedigree_info,sire_id,dam_id,dam_sire_id
0,497243.0,704422,0.00,b g,66932,94073,302525
1,497243.0,700589,2.50,b g,54398,521044,47456
2,497243.0,700253,2.75,b g,469953,563900,57937
3,497243.0,680613,27.75,b g,14183,680612,42830
4,497243.0,727829,43.75,ch g,49160,107101,304449
...,...,...,...,...,...,...,...
1867790,783151.0,2023661,18.25,ch m,695767,716793,541314
1867791,783151.0,2018833,19.00,b m,790757,706028,509320
1867792,783151.0,2755558,20.00,b f,663605,851467,465239
1867793,783151.0,3134051,23.00,bb f,793060,3134050,500508
